## Imporing

In [1]:
import pandas as pd
import os
from sklearn.preprocessing import MultiLabelBinarizer
import re
from collections import Counter
import re
from rapidfuzz import process, fuzz



## Preprocessing

In [2]:

# Load the Excel file
file_path = '20 obs - 13.04.25/iridology-v20.xlsx'  # Update with your actual path
df = pd.read_excel(file_path)

In [3]:

# Display basic information
print("📊 Basic Info:")
print(df.info())


📊 Basic Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 6 columns):
 #   Column                                     Non-Null Count  Dtype 
---  ------                                     --------------  ----- 
 0   Image ID                                   20 non-null     int64 
 1   Side                                       20 non-null     object
 2   Main Category                              20 non-null     object
 3   Sub-Category                               20 non-null     object
 4   Analysis(*)                                20 non-null     object
 5   (*) Overall conclusion and recommendation  20 non-null     object
dtypes: int64(1), object(5)
memory usage: 1.1+ KB
None


In [4]:

# Display first few rows
print("\n🔍 First 5 rows:")
df.head()



🔍 First 5 rows:


,Image ID,Side,Main Category,Sub-Category,Analysis(*),(*) Overall conclusion and recommendation
0,1,Right,Cardiovascular system\nNervous system\nEndocri...,Arteriosclerosis\nAnxiety\nPancreas weakness,Presence of a stress ring indicates anxiety. P...,Support nerves with magnesium and calming herb...
1,2,Left,Nervous system\nCirculatory system,Emotional stress \nAnxiety\nVein and lymph con...,The presence of radial furrows and a stress ri...,"Reduce stress through mindfulness, exercise, a..."
2,3,Right,Endocrine system\nNervous system\nRespiratory ...,Pancreas weakness\nInsulin resistance\nNeurolo...,"The body displays hormonal imbalances, particu...","Balance blood sugar with diet, herbs (cinnamon..."
3,4,Right,Endocrine System\nCardihjovascular system\nDig...,Adrenal exhustion\nBlood Circulation problem ...,A cholesterol ring suggests long-term cardiova...,"Take omega-3s for heart health, vitamin C and ..."
4,5,Right,Circulatory System\nEndocrine System\nNervous ...,Heart\nOvaries / Hormonal Balance Vitality\n,"Signs of stress on the heart, possibly from em...",Calm the nervous system with magnesium and dee...


In [5]:

# Check for missing values
print("\n❓ Missing values:")
print(df.isnull().sum())



❓ Missing values:
Image ID                                     0
Side                                         0
Main Category                                0
Sub-Category                                 0
Analysis(*)                                  0
(*) Overall conclusion and recommendation    0
dtype: int64


In [6]:

# Check for unique values per column
print("\n🔢 Unique values per column:")
for col in df.columns:
    print(f"{col}: {df[col].nunique()}")


🔢 Unique values per column:
Image ID: 20
Side: 2
Main Category: 20
Sub-Category: 20
Analysis(*): 20
(*) Overall conclusion and recommendation: 20


In [7]:
# Example: Current column names
print("📋 Original Columns:")
print(df.columns)

# Rename columns using a dictionary: {'old_name': 'new_name'}
df.rename(columns={
    'Image ID': 'image_id',
    'Side': 'side',
    'Main Category' : 'main_category',
    'Sub-Category' : 'sub_category',
    'Analysis(*)' : 'analysis',
    '(*) Overall conclusion and recommendation' : 'overall_recommendation'

    # Add more mappings as needed
}, inplace=True)

# Show updated column names
print("\n✅ Updated Columns:")
print(df.columns)


📋 Original Columns:
Index(['Image ID', 'Side', 'Main Category', 'Sub-Category', 'Analysis(*)',
       '(*) Overall conclusion and recommendation'],
      dtype='object')

✅ Updated Columns:
Index(['image_id', 'side', 'main_category', 'sub_category', 'analysis',
       'overall_recommendation'],
      dtype='object')


In [8]:
# Folder where your images are stored
image_folder = '20 obs - 13.04.25/pictures'  # Update this pat

In [9]:

# List of image filenames (just the numbers without '.jpg')
image_files = [f.split('.')[0] for f in os.listdir(image_folder) if f.endswith('.jpg')]

image_files


['8',
 '9',
 '14',
 '15',
 '17',
 '16',
 '12',
 '13',
 '11',
 '10',
 '20',
 '18',
 '19',
 '4',
 '5',
 '7',
 '6',
 '2',
 '3',
 '1']

In [10]:
image_files = sorted([int(f) for f in image_files])  # Convert to integers if needed
image_files

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [11]:
# Check what column in the dataframe can match these image IDs
print("\n📂 Image Files Found:")
print(image_files)


📂 Image Files Found:
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]


In [12]:
df.head()

,image_id,side,main_category,sub_category,analysis,overall_recommendation
0,1,Right,Cardiovascular system\nNervous system\nEndocri...,Arteriosclerosis\nAnxiety\nPancreas weakness,Presence of a stress ring indicates anxiety. P...,Support nerves with magnesium and calming herb...
1,2,Left,Nervous system\nCirculatory system,Emotional stress \nAnxiety\nVein and lymph con...,The presence of radial furrows and a stress ri...,"Reduce stress through mindfulness, exercise, a..."
2,3,Right,Endocrine system\nNervous system\nRespiratory ...,Pancreas weakness\nInsulin resistance\nNeurolo...,"The body displays hormonal imbalances, particu...","Balance blood sugar with diet, herbs (cinnamon..."
3,4,Right,Endocrine System\nCardihjovascular system\nDig...,Adrenal exhustion\nBlood Circulation problem ...,A cholesterol ring suggests long-term cardiova...,"Take omega-3s for heart health, vitamin C and ..."
4,5,Right,Circulatory System\nEndocrine System\nNervous ...,Heart\nOvaries / Hormonal Balance Vitality\n,"Signs of stress on the heart, possibly from em...",Calm the nervous system with magnesium and dee...


In [13]:
# Assuming there’s a column like 'ID' or 'image_id' in the Excel
print("\n🧾 Unique values from DataFrame column to match:")
print(df['image_id'].unique())  # Replace 'image_id' with actual column name if different


🧾 Unique values from DataFrame column to match:
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]


### Keeping only relavent columns for now 

In [14]:
df = df[['image_id', 'sub_category', 'main_category', 'side']]

In [15]:
df.head()

,image_id,sub_category,main_category,side
0,1,Arteriosclerosis\nAnxiety\nPancreas weakness,Cardiovascular system\nNervous system\nEndocri...,Right
1,2,Emotional stress \nAnxiety\nVein and lymph con...,Nervous system\nCirculatory system,Left
2,3,Pancreas weakness\nInsulin resistance\nNeurolo...,Endocrine system\nNervous system\nRespiratory ...,Right
3,4,Adrenal exhustion\nBlood Circulation problem ...,Endocrine System\nCardihjovascular system\nDig...,Right
4,5,Heart\nOvaries / Hormonal Balance Vitality\n,Circulatory System\nEndocrine System\nNervous ...,Right


### Labeling

In [16]:
[i for i in df['main_category']]

['Cardiovascular system\nNervous system\nEndocrine system',
 'Nervous system\nCirculatory system',
 'Endocrine system\nNervous system\nRespiratory system',
 'Endocrine System\nCardihjovascular system\nDigestive System',
 'Circulatory System\nEndocrine System\nNervous System',
 'Digestive System\nCirculatory System\nMusculoskeletal system',
 'Circulatory System\nDigestive System\nEndocrine/Elimination',
 'Endocrine system\nDigestive system\nNervous System',
 'Nervous System\nEndocrine System\nMusculoskeletal system',
 'Musculoskeletal System\nDigestive System\nReproductive System',
 'Respiratory / Lymphatic System\nDigestive System\nCirculatory System',
 'Digestive System\nImmune / Lymphatic System\nNervous System',
 'Circulatory system\nLymphatic / Immune system\nUrinary system',
 'Digestive system\nEndocrine system\nMusculoskeletal system',
 'Nervous  system\nDigestive system\nNervous/Endocrine system',
 'Nervous system\nDigestive system\nUrinary system',
 'Endocrine system\nRespirato

In [17]:

import re

def clean_main_category_text(text):
    # 1. Split original string on newlines first
    raw_labels = str(text).split('\n')
    final_labels = []

    for label in raw_labels:
        label = label.lower().strip()

        # 2. Split compound labels using /, &, or "and" (word boundary)
        # This regex splits on all 3 connectors
        parts = re.split(r'\s*(?:and|/|&)\s*', label)

        for part in parts:
            part = part.strip()
            if not part:
                continue

            # 3. If it doesn’t already contain “system”, append it
            if not part.endswith("system"):
                part += " system"

            # 4. Clean extra spaces
            part = re.sub(r'\s+', '_', part)

            final_labels.append(part)

    return list(set(final_labels))  # remove duplicates


df['main_category'] = df['main_category'].apply(clean_main_category_text)


In [18]:
# Flatten the list
all_main_labels = [label for sublist in df['main_category'] for label in sublist]

# Count each label
main_label_counts = Counter(all_main_labels)

# Sort by frequency (optional)
main_label_counts = dict(sorted(main_label_counts.items(), key=lambda x: x[1], reverse=True))

# Display
print("🧾 main_category label counts:")
for label, count in main_label_counts.items():
    print(f"{label:35} → {count}")

print("\n✅ Total unique labels:", len(main_label_counts))


🧾 main_category label counts:
nervous_system                      → 12
endocrine_system                    → 12
digestive_system                    → 10
circulatory_system                  → 8
musculoskeletal_system              → 5
respiratory_system                  → 4
lymphatic_system                    → 4
reproductive_system                 → 3
immune_system                       → 2
urinary_system                      → 2
cardiovascular_system               → 1
cardihjovascular_system             → 1
elimination_system                  → 1
vitality_zone_system                → 1

✅ Total unique labels: 14


In [19]:
# Step 1: Define the mapping dictionary
label_fix_map = {
    'cardihjovascular_system': 'cardiovascular_system',
    'elimination_system': 'endocrine_system',
    'vitality_zone_system': 'vitality_system'
}

# Step 2: Apply it to each list of labels
def apply_label_fix(label_list):
    return [label_fix_map.get(label, label) for label in label_list]

df['main_category'] = df['main_category'].apply(apply_label_fix)


In [20]:
from collections import Counter

all_main_labels = [label for sublist in df['main_category'] for label in sublist]
main_label_counts = dict(Counter(all_main_labels))
main_label_counts = dict(sorted(main_label_counts.items(), key=lambda x: x[1], reverse=True))

print("\n📊 main_category label counts after mapping:")
for label, count in main_label_counts.items():
    print(f"{label:35} → {count}")

print("\n✅ Total unique labels after fix:", len(main_label_counts))



📊 main_category label counts after mapping:
endocrine_system                    → 13
nervous_system                      → 12
digestive_system                    → 10
circulatory_system                  → 8
musculoskeletal_system              → 5
respiratory_system                  → 4
lymphatic_system                    → 4
reproductive_system                 → 3
cardiovascular_system               → 2
immune_system                       → 2
urinary_system                      → 2
vitality_system                     → 1

✅ Total unique labels after fix: 12
